<a href="https://colab.research.google.com/github/samaco634/deep-learning-basics/blob/main/01-1_%ED%82%A4%EC%99%80_%EB%AA%B8%EB%AC%B4%EA%B2%8C_%EC%84%A0%ED%98%95%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

출처 : https://skettee.github.io/post/linear_regression/

https://www.kaggle.com/datasets/burnoutminer/heights-and-weights-dataset

# 선형 회귀 (Linear Regression)

Date: 2019-08-15  
Author: skettee  
Categories: Deep Learning, Linear Regression  
Tags: Parameter, Loss function, Mean-squared error, Gradient Descent, Normalization
<!--eofm-->


딥러닝의 세계로 들어가기 위해 알아야 하는 첫번째 모델인 선형 회귀(Linear Regression)에 대해 알아보고 keras를 이용해서 모델링을 해보자!


## 문제 (Problem)

💰 고객  
 
> 우리 학교에는 '몸짱반'이 있어요.   
> 여기에 들어가기 위해서는   
> 키와 몸무게의      
> 특별한 조건이 있는거 같아요.  
>
> '몸짱반'에 들어간 친구들의   
> 키와 몸무게 데이터를 가지고   
> 키가 주어지면   
> '몸짱반'에 들어가기 위한 몸무게를  
> 자동으로 알고 싶어요...  
>
> 만들어 줄 수 있지요? 
>
> 데이터는 아래에 있어요  
> https://skettee.github.io/post/linear_regression/momjjangban_data.csv



## 데이터 수집 (Data Collection)

데이터를 다운로드하고 데이터프레임으로 변환한다.  

Keras의 get_file을 이용해서 데이터를 다운로드 한다.

In [ ]:
from tensorflow.keras.utils import get_file

fname = 'momjjangban_data.csv'
origin = 'https://skettee.github.io/post/linear_regression/momjjangban_data.csv'
path = get_file(fname, origin)

CSV파일을 읽어서 데이터프레임으로 변환한다.  

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv(path, index_col=0)

df.head()

## 데이터 클렌징 (Data Cleansing)

데이터가 빵꾸난 것이 없는지 확인한다. 빵꾸난 데이터는 제거한다.  

⚙️ 엔지니어

> 얼굴만 클렌징하는거 아니다...  
> 데이터도 클렌징해야 한다.  

In [ ]:
# Check nan
df.isnull().any()

In [ ]:
# Remove row with nan
df = df.dropna(axis=0).reset_index(drop=True)

df.head()

In [ ]:
height_data = df.height
weight_data = df.weight

print(height_data.shape)
print(weight_data.shape)

## 데이터 분석 (Data Analysis)

데이터가 어떤 모양인지 확인해 보자.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

plt.scatter(height_data, weight_data)
plt.xlabel('height (cm)')
plt.ylabel('weight (kg)')
plt.show()

⚙️ 엔지니어

> 키가 커질 수록 몸무게가 선형(Linear)적으로 늘어나네~   
> 직선으로 모델링을 할 수 있겠다.
> 
> 모델링 작업이 편안하도록 주어진 데이터를 변환하자!

## 데이터 변환 (Data Transformation)

Keras, tensorflow등의 머신런닝 프레임워크를 사용하기 위해서는  
각각이 요구하는 데이터의 형식 및 모양에 맞추어서 변환을 해 주어야 한다.  
여기서는  키와 몸무게 데이터를 각각 매트릭스로 변환하자  

- 열의 크기는 데이터의 개수      
- 컬럼의 크기는 측정한 항목의 개수   
 
키 값을 입력하면 '몸짱반'에 들어갈 수 있는 몸무게를 예측해야 한다. 키 데이터를 입력 $x$라고 하고 몸무게 데이터를 출력 $y$라고 하자 

- 키 데이터는 50개의 '키'를 측정한 데이터가 있으므로 50X1 매트릭스   
- 몸무게 데이터는 50개의 '몸무게'를 측정한 데이터가 있으므로 50X1 매트릭스이다.  

⚙️ 엔지니어

> 모델링을 하면서 골치 아픈 것 중에 하나는  
> 입력과 출력의 모양(Shape)를 맞추어 주는 것이다.  
> 복잡하다 ... 잘못하면 다크-디멘션에 빠진다...   
> 뒤에서 더 자세하게 설명할 기회가 있을 것이다.  



In [ ]:
print('x = ', height_data[:10])
print('x.shape= ', height_data.shape)
print('\ny = ', weight_data[:10])
print('y.shape= ', weight_data.shape)

In [ ]:
x = np.array(height_data).reshape(len(height_data), 1)
y = np.array(weight_data).reshape(len(weight_data), 1)

print('x = ', x[:10])
print('x.shape= ', x.shape)
print('\ny = ', y[:10])
print('y.shape= ', y.shape)

⚙️ 엔지니어

> 좋았어!   
> 
> 그럼 변환된 데이터로 선형회귀(Linear Regression) 모델링을 시작해 보지

## 선형 모델링 (Linear Modeling)

모델을 $y=wx+b$로 놓고   
주어진 데이터에 가장 비슷한 직선을 표시하는 $w$와 $b$를 찾아 가는 것이 선형 모델링이다.  
$w$는 직선의 기울기고 $b$는 $y$절편이라고 배웠지만, 이제 부터는 $w$는 **가중치(weight)** 로, $b$는 **바이어스(bias)** 로 부르자. 그리고 $w, b$를 선형 모델의 **파라미터(parameter)** 라고 한다.


## 케라스(Keras)로 모델링(Modeling)

⚙️ 엔지니어

> 좋았어!   
> 
> 이제 텐서플로우(Tensorflow)가 냠냠한   
> **케라스(Keras)**를 이용해서 구현을 해보자!

### 정규화(Normalization)
  
`정규값 = (현재값 - 최소값) / (최대값-최소값)` 으로 정규화 하자  
 
그래프를 보면,  
데이터의 모양은 그대로 유지하면서도  
x축의 값이 0에서 1사이로 변환된 것을 볼 수 있다.  

⚙️ 엔지니어  

> 내가 해봐서 아는데...   
> 
> 이거 안하면 엉뚱한 답이 나온다.
> 꼭 해야됨!



In [ ]:
from sklearn import preprocessing

mm_scaler = preprocessing.MinMaxScaler()
X_train = mm_scaler.fit_transform(x)
Y_train = mm_scaler.transform(y)

plt.scatter(X_train, Y_train)
plt.xlabel('scaled-height')
plt.ylabel('scaled-weight')
plt.show()

### 모델링 (Modeling)

⚙️ 엔지니어

> 4줄로 모델링이 가능하다!   
> 
> 케라스 만세!



In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

# 모델을 준비한다.
model = Sequential()

# 입력 변수의 개수가 1이고 출력 개수가 1인 y=wx+b 를 생성한다.
model.add(Dense(1, input_dim=1))

# Loss funtion과 Optimizer를 선택한다.
model.compile(loss='mean_squared_error', optimizer='sgd') 

model.summary()

In [ ]:
# epochs만큼 반복해서 손실값이 최저가 되도록 모델을 훈련한다.
hist = model.fit(X_train, Y_train, epochs=300, verbose=1) 

### 손실값의 변화를 그래프로 확인

In [ ]:
plt.plot(hist.history['loss'])
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

⚙️ 엔지니어

> 반복적으로 학습할 수록 손실(Loss)이 0에 가깝게 된다.  
> 굿잡!

### $w$와 $b$값을 확인

In [ ]:
w, b = model.get_weights()
w =  w[0][0]
b = b[0]
print('w: ', w)
print('b: ', b)

### 그래프로 확인

In [ ]:
x_scale = mm_scaler.transform(x)
y_scale = mm_scaler.transform(y)
plt.scatter(x_scale, y_scale)
plt.plot(x_scale, w*x_scale+b, 'r')
plt.xlabel('scaled-height')
plt.ylabel('scaled-weight')
plt.show()

## 학습 횟수를 늘려 봅시다

In [ ]:
# epochs만큼 반복해서 손실값이 최저가 되도록 모델을 훈련한다.
hist = model.fit(X_train, Y_train, epochs=1000, verbose=1) 

In [ ]:
w, b = model.get_weights()
w =  w[0][0]
b = b[0]
print('w: ', w)
print('b: ', b)

In [ ]:
x_scale = mm_scaler.transform(x)
y_scale = mm_scaler.transform(y)
plt.scatter(x_scale, y_scale)
plt.plot(x_scale, w*x_scale+b, 'r')
plt.xlabel('scaled-height')
plt.ylabel('scaled-weight')
plt.show()

## 해결 (Solution)

⚙️ 엔지니어  

> 고객님~ 원하시는 솔루션입니다.   
> input_height에 원하시는 키를 입력하시면    
> '몸짱반'에 들어 갈 수 있는 몸무게가 자동으로 계산됩니다.    



In [ ]:
input_height = 178.0

input_x = mm_scaler.transform(np.array([input_height]).reshape(-1, 1))
predict = model.predict(input_x)
predict = mm_scaler.inverse_transform(predict)

print('몸짱반에 들어갈 수 있는 몸무게는 {:.2f} kg 입니다'.format(predict[0][0]))